# Coupled-cluster doubles

In this notebook we set up a general solver for CCD given matrices $h_{p}^{q}$ and $u^{pq}_{rs}$, where the latter is _anti-symmetric_.

In [10]:
import pickle
import numpy as np

We start by retrieving the pickled one-body and two-body matrix elements generated in the `matrix_elements.ipynb`-notebook.

In [9]:
h = pickle.load(open("dat/h.pkl", "rb"))
u = pickle.load(open("dat/u.pkl", "rb"))